In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time

C:\ProgramData\Anaconda3\envs\Tensor3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\Tensor3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\Tensor3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\Tensor3\lib\site-packages\tensorflow\

### 1 加载数据集

In [2]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

W0818 20:12:56.870435  7832 deprecation.py:323] From <ipython-input-2-83231f068ae1>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0818 20:12:56.873435  7832 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0818 20:12:56.875436  7832 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruc

Extracting MNIST_data\train-images-idx3-ubyte.gz


W0818 20:12:57.124450  7832 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0818 20:12:57.127450  7832 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0818 20:12:57.172453  7832 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\Tensor3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecat

Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


### 2 设定批次

In [3]:
batch_size = 50
n_batch = mnist.train.num_examples // batch_size  # 总共有多少个批次

# 定义训练集占位符
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='input_x')  # 每一行表示一个样本，内容是784为的像素值
    y = tf.placeholder(tf.float32, [None, 10], name='input_y')   # 每一行表示一个label，采用杜热编码
keep_prob = tf.placeholder(tf.float32)

### 3 创建安神经网络（不含隐藏层）

In [4]:
with tf.name_scope('BPNN'):    
    w1 = tf.Variable(tf.truncated_normal([784, 1000], stddev=0.1))
    b1 = tf.Variable(tf.zeros([1000]) + 0.1)
    wx_plus_b_l1 = tf.matmul(x, w1) + b1
    l1 = tf.nn.tanh(wx_plus_b_l1)
    l1_dropout = tf.nn.dropout(l1, keep_prob)

    w2 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.1))
    b2 = tf.Variable(tf.zeros([10]) + 0.1)
    wx_plus_b_l2 = tf.matmul(l1_dropout, w2) + b2
    l2 = tf.nn.tanh(wx_plus_b_l2)
    prediction = tf.nn.dropout(l2, keep_prob)

# w3 = tf.Variable(tf.truncated_normal([1500, 1000], stddev=0.1))
# b3 = tf.Variable(tf.zeros([1000]) + 0.1)
# wx_plus_b_l3 = tf.matmul(l2_dropout, w3) + b3
# l3 = tf.nn.tanh(wx_plus_b_l3)
# l3_dropout = tf.nn.dropout(l3, keep_prob)

# w4 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.1))
# b4 = tf.Variable(tf.zeros([10]) + 0.1)
# wx_plus_b_l4 = tf.matmul(l3_dropout, w4) + b4
# l4 = tf.nn.tanh(wx_plus_b_l4)
# prediction = tf.nn.dropout(l4, keep_prob)

W0818 20:12:57.559475  7832 deprecation.py:506] From <ipython-input-4-5cf43015ac0e>:6: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### 4 优化器

In [5]:
# train_step = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss)
    
with tf.name_scope('train_step'):
    train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

W0818 20:12:57.626479  7832 deprecation.py:323] From <ipython-input-5-ef0f113342f4>:3: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 5 获取预测结果并计算准确率

In [6]:
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.arg_max(prediction, 1), tf.arg_max(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

W0818 20:12:57.778487  7832 deprecation.py:323] From <ipython-input-6-b86b710cb5ec>:2: arg_max (from tensorflow.python.ops.gen_math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.math.argmax` instead


### 6 启动会话

In [7]:
merged = tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('G:/tensorflow/logs/',sess.graph)
    for epoch in range(21):
        time1 = time.time()
        for batch in range(n_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, summary = sess.run([train_step, merged], feed_dict={x:batch_x, y:batch_y, keep_prob:1.0})     
        writer.add_summary(summary, epoch)
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0})  # 用测试集来预测
        print("第{0}圈  准确率是： {1}, 耗时: {2} 秒 ".format(epoch, acc, time.time()-time1))

第0圈  准确率是： 0.9373000264167786, 耗时: 6.285359621047974 秒 
第1圈  准确率是： 0.9569000005722046, 耗时: 6.920395851135254 秒 
第2圈  准确率是： 0.9625999927520752, 耗时: 6.997400283813477 秒 
第3圈  准确率是： 0.9707000255584717, 耗时: 7.069404125213623 秒 
第4圈  准确率是： 0.9735000133514404, 耗时: 6.483370780944824 秒 
第5圈  准确率是： 0.9735000133514404, 耗时: 6.348363161087036 秒 
第6圈  准确率是： 0.9732999801635742, 耗时: 6.161352634429932 秒 
第7圈  准确率是： 0.9776999950408936, 耗时: 6.231356382369995 秒 
第8圈  准确率是： 0.9782000184059143, 耗时: 6.213355302810669 秒 
第9圈  准确率是： 0.9754999876022339, 耗时: 6.560375213623047 秒 
第10圈  准确率是： 0.9789000153541565, 耗时: 6.245357275009155 秒 
第11圈  准确率是： 0.9786999821662903, 耗时: 6.491371393203735 秒 
第12圈  准确率是： 0.979200005531311, 耗时: 6.229356288909912 秒 
第13圈  准确率是： 0.978600025177002, 耗时: 6.770387411117554 秒 
第14圈  准确率是： 0.9783999919891357, 耗时: 6.495371580123901 秒 
第15圈  准确率是： 0.9811000227928162, 耗时: 6.3973658084869385 秒 
第16圈  准确率是： 0.9796000123023987, 耗时: 6.418367147445679 秒 
第17圈  准确率是： 0.9775999784469604, 耗时: 6.1623